**Load needed libraries**

In [ ]:
#import libraries
import pandas as pd    # Importing the pandas library, which is used for data manipulation and analysis, particularly with DataFrames.
import statsmodels.api as sm    # Importing statsmodels library for statistical modeling and hypothesis testing.
import sklearn as sk    # Importing scikit-learn (sklearn), a popular machine learning library for data preprocessing, model building, and evaluation.
from statsmodels.stats.outliers_influence import variance_inflation_factor    # Importing a method to calculate Variance Inflation Factor (VIF) to check for multicollinearity in features.
from sklearn.model_selection import train_test_split, cross_val_score    # Importing methods for splitting data into training/testing sets and performing cross-validation.
from sklearn.linear_model import LinearRegression    # Importing LinearRegression class to create a linear regression model.
from sklearn.metrics import mean_squared_error, mean_absolute_error    # Importing methods to evaluate model performance using mean squared error and mean absolute error.
from sklearn.feature_selection import SequentialFeatureSelector    # Importing SequentialFeatureSelector for feature selection using a greedy approach.
from sklearn.pipeline import make_pipeline    # Importing make_pipeline to chain multiple steps (preprocessing, modeling) into one pipeline.
from sklearn.linear_model import Ridge    # Importing Ridge regression, a type of linear regression that applies L2 regularization.
from sklearn.preprocessing import StandardScaler    # Importing StandardScaler to standardize features by removing the mean and scaling to unit variance.
from sklearn.linear_model import Lasso    # Importing Lasso regression, a type of linear regression that applies L1 regularization to perform feature selection.
from sklearn.linear_model import ElasticNet    # Importing ElasticNet regression, a combination of Lasso and Ridge regression that applies both L1 and L2 regularization.
import numpy as np    # Importing numpy library for numerical operations, especially with arrays and matrices.
import matplotlib.pyplot as plt    # Importing matplotlib's pyplot module for creating visualizations like plots and graphs.
import seaborn as sns    # Importing seaborn, a statistical data visualization library built on top of matplotlib for creating more attractive and informative plots.
from statsmodels.stats.stattools import durbin_watson    # Importing the Durbin-Watson statistic function to check for autocorrelation in residuals of a regression model.


**Load the data from a csv file**

In [ ]:
df = pd.read_excel('college_admissions_2022.xlsx')

**Print Info**

In [ ]:
print(df.info)

**Print Descriptive Statistics**

In [ ]:

pd.set_option('display.max_columns', None)
display(df)

**Drop the missing values**

In [ ]:
# Drop missing values
df_cleaned = df.dropna()

# Report the size of the remaining dataset
remaining_size = df_cleaned.shape
print(df_cleaned.info())
print(f"Size of the remaining dataset (rows, columns): {remaining_size}")

In [ ]:
pd.set_option('display.max_columns', None)
display(df_cleaned)

**Remove the variables, 'UNITID', 'INSTNM', 'Appl'  from the dataset. And split the data set into a training set (60%) and a test set (40%) and use random_state=101.**

In [ ]:
# Remove the specified variables
df_cleaned_columns = df_cleaned.drop(columns=['UNITID', 'INSTNM', 'Appl'])
print(df_cleaned_columns.info())



In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)


print(f"Training set size (rows, columns): {X_train.shape}")
print(f"Test set size (rows, columns): {X_test.shape}")

**Fit a linear model using least squares on the training set with all other remaining variables. Report the coefficients, and the test errors, ME, RMSE, MAE, MPE, and MAPE of the validation part.**

In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']
#y = np.log(df_cleaned_columns['Ug_enter'])

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Add a constant to the model (intercept term)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Fit a linear model using least squares on the training set
linear_model = sm.OLS(y_train, X_train_const).fit()

# Predict on the test set
y_pred = linear_model.predict(X_test_const)

# Calculate validation errors
ME = np.mean(y_pred - y_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
MAE = mean_absolute_error(y_test, y_pred)
MPE = np.mean((y_pred - y_test) / y_test) * 100
MAPE = np.mean(np.abs((y_pred - y_test) / y_test)) * 100

# Coefficients
coefficients = linear_model.params
p_values = linear_model.pvalues
r_squared = linear_model.rsquared
adj_r_squared = linear_model.rsquared_adj
f_statistic = linear_model.fvalue

# Calculate AIC and BIC
aic = linear_model.aic
bic = linear_model.bic

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Create a DataFrame to display variable names, coefficients, p-values, and VIF
coefficients_df = pd.DataFrame({
    'Variable': X_train_const.columns,
    'Coefficient': coefficients.values,
    'P-Value': p_values.values,
    'VIF': vif_data["VIF"]
})

# Print results
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print()
print("Model Error Metrics:")
print("Mean Error (ME):", ME)
print("Root Mean Square Error (RMSE):", RMSE)
print("Mean Absolute Error (MAE):", MAE)
print("Mean Percentage Error (MPE):", MPE)
print("Mean Absolute Percentage Error (MAPE):", MAPE)
print()
print("Model Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

linear_ME = ME
liner_RMSE = RMSE
linear_MAE = MAE
linear_MPE = MPE
linear_MAPE = MAPE
linear_r_squared = r_squared
linear_adjusted_r_squared = adj_r_squared
linear_f_statistic = f_statistic
linear_aic = aic
linear_bic = bic

# Residual Analysis
residuals = y_test - y_pred

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Linear Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()


**Perform backward elimination on the dataset and fit a linear regression model. Report the test error obtained. Which variables were retained in the final model? Additionally, provide the validation errors, including Mean Error (ME), Root Mean Square Error (RMSE), Mean Absolute Error (MAE), Mean Percentage Error (MPE), and Mean Absolute Percentage Error (MAPE).**

Backward regression (or backward elimination) is a stepwise feature selection technique used in linear regression to build an optimal model by removing the least significant predictors one at a time. It starts with all available independent variables in the model. The algorithm then evaluates the statistical significance of each predictor, typically using p-values from t-tests. If the p-value of the least significant variable exceeds a predefined threshold (e.g., 0.05), that variable is removed from the model. This process is repeated iteratively until only statistically significant predictors remain in the model, ensuring a simpler, more interpretable model with reduced overfitting.

Multicollinearity occurs in a regression model when two or more independent variables (predictors) are highly correlated with each other. This means that one predictor can be linearly predicted from the others with a high degree of accuracy, leading to redundant information in the model.

**Multicollinearity can cause problems in regression analysis, such as:**

Unstable Coefficients: It makes the coefficients of the predictors sensitive to small changes in the data, leading to large variances and unreliable estimates.

Interpretation Issues: It becomes difficult to assess the individual effect of each predictor because the variables overlap in the information they provide.

Increased Standard Errors: This can lead to statistically insignificant p-values even for predictors that are important.

Although multicollinearity doesn't affect the model's overall predictive power, it complicates interpretation and can lead to less precise models. Techniques like Ridge Regression or Variance Inflation Factor (VIF) are often used to detect and address multicollinearity.

In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']
#y = np.log(df_cleaned_columns['Ug_enter'])

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Add a constant to the model (intercept term)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Perform backward elimination
model = sm.OLS(y_train, X_train_const).fit()
p_values = model.pvalues

dropped_variables = []

# Iteratively remove variables with the highest p-value greater than 0.05
while max(p_values) > 0.05:
    dropped_var = p_values.idxmax()
    dropped_variables.append(dropped_var)
    X_train_const = X_train_const.drop(columns=[dropped_var])
    X_test_const = X_test_const.drop(columns=[dropped_var])
    model = sm.OLS(y_train, X_train_const).fit()
    p_values = model.pvalues

# Final model
final_model = model
y_pred = final_model.predict(X_test_const)

# Calculate validation errors for backward elimination model
ME = np.mean(y_pred - y_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
MAE = mean_absolute_error(y_test, y_pred)
MPE = np.mean((y_pred - y_test) / y_test) * 100
MAPE = np.mean(np.abs((y_pred - y_test) / y_test)) * 100

# Coefficients
coefficients = final_model.params
p_values = final_model.pvalues
r_squared = final_model.rsquared
adj_r_squared = final_model.rsquared_adj
f_statistic = final_model.fvalue

# Calculate AIC and BIC
aic = final_model.aic
bic = final_model.bic

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Create a DataFrame to display variable names, coefficients, p-values, and VIF
coefficients_df = pd.DataFrame({
    'Variable': X_train_const.columns,
    'Coefficient': coefficients.values,
    'P-Value': p_values.values,
    'VIF': vif_data["VIF"]
})

# Print results
print("Dropped Variables:")
for variable in dropped_variables:
    print(variable)
print()
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print()
print("Model Error Statistics:")
print("Mean Error (ME):", ME)
print("Root Mean Square Error (RMSE):", RMSE)
print("Mean Absolute Error (MAE):", MAE)
print("Mean Percentage Error (MPE):", MPE)
print("Mean Absolute Percentage Error (MAPE):", MAPE)
print()
print("Model Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

backward_ME = ME
backward_RMSE = RMSE
backward_MAE = MAE
backward_MPE = MPE
backward_MAPE = MAPE
backward_r_squared = r_squared
backward_adjusted_r_squared = adj_r_squared
backward_f_statistic = f_statistic
backward_aic = aic
backward_bic = bic

# Residual Analysis
residuals = y_test - y_pred

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Backward Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()

**Fit a forward selection and report the test error obtained. Which variables did you end up with? Which variables did you end up with? What are the validation errors (ME, RMSE, MAE, MPE, and MAPE )**


Forward regression (or forward selection) is a feature selection technique where the model starts with no predictors and adds them one by one based on their statistical significance. At each step, the predictor that improves the model the most (typically by minimizing the error or based on a p-value threshold) is added. This process continues until adding more predictors no longer significantly improves the model.

The key difference between forward regression and stepwise regression is that in stepwise regression, after each step (either adding or removing a predictor), the model also checks whether previously included predictors should be removed based on their significance. Stepwise regression is more flexible because it combines elements of both forward and backward selection, whereas forward regression only adds predictors without removing any previously added ones.

In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Perform forward selection
linear_model = LinearRegression()
sfs = SequentialFeatureSelector(linear_model, direction='forward', scoring='neg_mean_squared_error', cv=5)
sfs.fit(X_train, y_train)

# Selected features
selected_features = sfs.get_support(indices=True)
X_train_fs = X_train.iloc[:, selected_features]
X_test_fs = X_test.iloc[:, selected_features]

# Fit the model using selected features
linear_model.fit(X_train_fs, y_train)
y_pred_fs = linear_model.predict(X_test_fs)

# Calculate validation errors for forward selection model
ME_fs = np.mean(y_pred_fs - y_test)
RMSE_fs = np.sqrt(mean_squared_error(y_test, y_pred_fs))
MAE_fs = mean_absolute_error(y_test, y_pred_fs)
MPE_fs = np.mean((y_pred_fs - y_test) / y_test) * 100
MAPE_fs = np.mean(np.abs((y_pred_fs - y_test) / y_test)) * 100

# Retained and dropped variables
initial_variables = X_train.columns.tolist()
retained_variables = X_train.columns[selected_features].tolist()
dropped_variables = [var for var in initial_variables if var not in retained_variables]

# Calculate VIF for each feature in the retained variables
X_train_const = sm.add_constant(X_train_fs)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Fit a linear model using statsmodels to get p-values, R-squared, adjusted R-squared, F-statistic, AIC, and BIC
linear_model_sm = sm.OLS(y_train, X_train_const).fit()
p_values = linear_model_sm.pvalues
r_squared = linear_model_sm.rsquared
adj_r_squared = linear_model_sm.rsquared_adj
f_statistic = linear_model_sm.fvalue
aic = linear_model_sm.aic
bic = linear_model_sm.bic

# Create a DataFrame to display variable names, coefficients, p-values, and VIF
coefficients_full = [linear_model.intercept_] + linear_model.coef_.tolist()
p_values_full = [None] + p_values.tolist()  # Add None for the intercept
coefficients_df = pd.DataFrame({
    'Variable': ['Intercept'] + retained_variables,
    'Coefficient': coefficients_full[:len(retained_variables) + 1],
    'P-Value': p_values_full[:len(retained_variables) + 1],
    'VIF': vif_data["VIF"]
})

# Adjust the lengths of the initial, retained, and dropped variables lists
max_length = max(len(initial_variables), len(retained_variables), len(dropped_variables))
initial_variables += [None] * (max_length - len(initial_variables))
retained_variables += [None] * (max_length - len(retained_variables))
dropped_variables += [None] * (max_length - len(dropped_variables))

# Create a DataFrame to show the initial, retained, and dropped variables
variables_df = pd.DataFrame({
    "Initial Variables": initial_variables,
    "Retained Variables": retained_variables,
    "Dropped Variables": dropped_variables
})

# Print results
print("Dropped Variables:")
for variable in dropped_variables:
    print(variable)
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print("\nValidation Errors:")
print("Mean Error (ME):", ME_fs)
print("Root Mean Square Error (RMSE):", RMSE_fs)
print("Mean Absolute Error (MAE):", MAE_fs)
print("Mean Percentage Error (MPE):", MPE_fs)
print("Mean Absolute Percentage Error (MAPE):", MAPE_fs)
print("\nModel Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

forward_ME = ME
forward_RMSE = RMSE
forward_MAE = MAE
forward_MPE = MPE
forward_MAPE = MAPE
forward_r_squared = r_squared
forward_adjusted_r_squared = adj_r_squared
forward_f_statistic = f_statistic
forward_aic = aic
forward_bic = bic

# Residual Analysis
residuals = y_test - y_pred_fs

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_fs, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_fs, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Forward Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()


**Run a Ridge regression with Standard Scaler, and alpha=0.5 and report the validation errors (ME, RMSE, MAE, MPE, and MAPE ).**


Ridge regression is a type of linear regression that includes a penalty term to reduce model complexity and prevent overfitting. It modifies the standard linear regression by adding a regularization term to the loss function, which is the L2 penalty (the square of the magnitude of coefficients). This penalty term discourages large coefficients, effectively shrinking them towards zero. By doing so, ridge regression balances between fitting the data well and maintaining smaller, more generalized coefficients, making it especially useful when there is multicollinearity or when the dataset has many features



In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Perform Ridge regression with Standard Scaler and alpha=0.5
ridge_model = make_pipeline(StandardScaler(), Ridge(alpha=0.5))
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

# Calculate cross-validation scores
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()

# Calculate validation errors for Ridge regression model
ME_ridge = np.mean(y_pred_ridge - y_test)
RMSE_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
MAE_ridge = mean_absolute_error(y_test, y_pred_ridge)
MPE_ridge = np.mean((y_pred_ridge - y_test) / y_test) * 100
MAPE_ridge = np.mean(np.abs((y_pred_ridge - y_test) / y_test)) * 100

# Get coefficients and corresponding variable names
coefficients = ridge_model.named_steps['ridge'].coef_
initial_variables = X_train.columns.tolist()

# Set a threshold to consider coefficients as dropped
threshold = 1e-5
retained_variables = [var for var, coef in zip(initial_variables, coefficients) if abs(coef) > threshold]
dropped_variables = [var for var, coef in zip(initial_variables, coefficients) if abs(coef) <= threshold]

# Calculate VIF for each feature in the retained variables
X_train_retained = X_train[retained_variables]
X_train_const = sm.add_constant(X_train_retained)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Fit a linear model using statsmodels to get p-values, AIC, and BIC
linear_model_sm = sm.OLS(y_train, X_train_const).fit()
p_values = linear_model_sm.pvalues
r_squared = linear_model_sm.rsquared
adj_r_squared = linear_model_sm.rsquared_adj
f_statistic = linear_model_sm.fvalue
aic = linear_model_sm.aic
bic = linear_model_sm.bic

# Create a DataFrame to display variable names, coefficients, p-values, and VIF
coefficients_full = [ridge_model.named_steps['ridge'].intercept_] + coefficients.tolist()
p_values_full = [None] + p_values.tolist()  # Add None for the intercept
coefficients_df = pd.DataFrame({
    'Variable': ['Intercept'] + retained_variables,
    'Coefficient': coefficients_full[:len(retained_variables) + 1],
    'P-Value': p_values_full[:len(retained_variables) + 1],
    'VIF': vif_data["VIF"]
})

# Print results
print("Dropped Variables:")
for variable in dropped_variables:
    print(variable)
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print("\nCross-Validation MSE:", cv_mse)
print("\nValidation Errors:")
print("Mean Error (ME):", ME_ridge)
print("Root Mean Square Error (RMSE):", RMSE_ridge)
print("Mean Absolute Error (MAE):", MAE_ridge)
print("Mean Percentage Error (MPE):", MPE_ridge)
print("Mean Absolute Percentage Error (MAPE):", MAPE_ridge)
print("\nModel Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

ridge_ME = ME_ridge
ridge_RMSE = RMSE_ridge
ridge_MAE = MAE_ridge
ridge_MPE = MPE_ridge
ridge_MAPE = MAPE_ridge
ridge_r_squared = r_squared
ridge_adjusted_r_squared = adj_r_squared
ridge_f_statistic = f_statistic
ridge_aic = aic
ridge_bic = bic

# Residual Analysis
residuals = y_test - y_pred_ridge

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_ridge, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Regularization Path
alphas = np.logspace(-6, 6, 200)
coefs = []
for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Ridge Regression Regularization Path')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_ridge, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Ridge Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()

**Run a Lasso regression with Standard Scaler, and alpha=0.5 and report the validation errors (ME, RMSE, MAE, MPE, and MAPE ).**

When analyzing a regularization path in the context of regression models (such as Lasso, Ridge, or Elastic Net regression), the convergence of coefficients toward specific values (like zero or a large value like 10,000) provides important insights into how regularization is affecting the model. Here’s a detailed explanation:

What is a Regularization Path?

	•	The regularization path shows how the coefficients of the regression model change as the regularization strength (or penalty term) is varied.
	•	In regularization, the goal is to control overfitting by shrinking the magnitude of the coefficients.
	•	As the regularization strength (controlled by a parameter like alpha or lambda) increases, coefficients are often shrunk towards zero (especially in Lasso or Elastic Net).

Convergence to Zero:

	•	Convergence to zero means that as the regularization parameter increases, the coefficients of the model are gradually shrinking to zero.
	•	This is typical behavior in Lasso regression (L1 regularization) and Elastic Net, where some coefficients can be driven exactly to zero.

Interpretation:

	1.	Smaller or Less Important Features:
	•	Features with coefficients that converge to zero are likely less important for predicting the target variable.
	•	In Lasso, regularization acts as a form of feature selection, setting some coefficients to zero, effectively removing those features from the model.
	2.	Highly Regularized Model:
	•	When most or all coefficients converge to zero, the model becomes highly regularized and might underfit the data because it is not using enough information from the features.
	3.	Simplicity of the Model:
	•	A model with coefficients converging to zero results in a simpler model with fewer features, which may generalize better (especially in the presence of noise or irrelevant features).

Convergence to a Large Value (e.g., 10,000):

	•	If coefficients converge to large values as the regularization path progresses, it indicates that the model is placing significant importance on certain features.

Interpretation:

	1.	Key Features:
	•	Coefficients that remain large (or grow) despite regularization are likely key predictors of the target variable.
	•	These features have a strong relationship with the target, and the model retains their influence even with stronger regularization.
	2.	Model with Lower Regularization:
	•	When coefficients remain large or don’t shrink much, the regularization is weaker, meaning the model is allowed to fit more complex relationships in the data.
	3.	Potential Overfitting:
	•	If regularization strength is too low, coefficients can become large and may lead to overfitting. The model may become too sensitive to the training data, capturing noise rather than general patterns.

Visualizing the Regularization Path:

	•	The regularization path plot often shows multiple lines, where each line represents the path of a feature’s coefficient as regularization increases.
	•	On the x-axis, the plot might show the value of the regularization parameter (like lambda), and on the y-axis, it shows the coefficient values.
	•	At low values of regularization (small lambda), coefficients tend to have larger magnitudes.
	•	At high values of regularization (large lambda), coefficients may converge to zero.

Convergence at Zero vs 10,000:

	•	Convergence at Zero: Indicates the feature’s effect is being reduced or eliminated entirely by regularization, suggesting it is less important or possibly irrelevant.
	•	Convergence at 10,000 (or any large value): Indicates the feature is highly influential, and its coefficient remains large because it plays a critical role in the model.

Example: Lasso Regularization Path

For Lasso regression, here’s what a regularization path might look like:

	•	Low Regularization Strength: At the start (low regularization), all coefficients are non-zero and large.
	•	Increasing Regularization: As regularization increases, some coefficients begin to shrink toward zero.
	•	High Regularization Strength: At high regularization values, many coefficients converge to zero, and only a few influential features retain non-zero coefficients.

Summary:

	•	Convergence to zero means regularization is shrinking the coefficients of less important features, reducing model complexity, and helping with feature selection (especially in Lasso and Elastic Net).
	•	Convergence to a large value (like 10,000) indicates that certain features remain important to the model despite the application of regularization. This typically means these features have a strong predictive power.
	•	The goal of regularization is to find a balance, ensuring that the model generalizes well without overfitting or underfitting the data.

In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Perform Ridge regression with Standard Scaler and alpha=0.5
ridge_model = make_pipeline(StandardScaler(), Ridge(alpha=0.5))
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

# Calculate cross-validation scores
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()

# Calculate validation errors for Ridge regression model
ME_ridge = np.mean(y_pred_ridge - y_test)
RMSE_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
MAE_ridge = mean_absolute_error(y_test, y_pred_ridge)
MPE_ridge = np.mean((y_pred_ridge - y_test) / y_test) * 100
MAPE_ridge = np.mean(np.abs((y_pred_ridge - y_test) / y_test)) * 100

# Get coefficients and corresponding variable names
coefficients = ridge_model.named_steps['ridge'].coef_
initial_variables = X_train.columns.tolist()

# Set a threshold to consider coefficients as dropped
threshold = 1e-5
retained_variables = [var for var, coef in zip(initial_variables, coefficients) if abs(coef) > threshold]
dropped_variables = [var for var, coef in zip(initial_variables, coefficients) if abs(coef) <= threshold]

# Calculate VIF for each feature in the retained variables
X_train_retained = X_train[retained_variables]
X_train_const = sm.add_constant(X_train_retained)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Fit a linear model using statsmodels to get p-values, AIC, and BIC
linear_model_sm = sm.OLS(y_train, X_train_const).fit()
p_values = linear_model_sm.pvalues
r_squared = linear_model_sm.rsquared
adj_r_squared = linear_model_sm.rsquared_adj
f_statistic = linear_model_sm.fvalue
aic = linear_model_sm.aic
bic = linear_model_sm.bic

# Create a DataFrame to display variable names, coefficients, p-values, and VIF
coefficients_full = [ridge_model.named_steps['ridge'].intercept_] + coefficients.tolist()
p_values_full = [None] + p_values.tolist()  # Add None for the intercept
coefficients_df = pd.DataFrame({
    'Variable': ['Intercept'] + retained_variables,
    'Coefficient': coefficients_full[:len(retained_variables) + 1],
    'P-Value': p_values_full[:len(retained_variables) + 1],
    'VIF': vif_data["VIF"]
})

# Print results
print("Dropped Variables:")
for variable in dropped_variables:
    print(variable)
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print("\nCross-Validation MSE:", cv_mse)
print("\nValidation Errors:")
print("Mean Error (ME):", ME_ridge)
print("Root Mean Square Error (RMSE):", RMSE_ridge)
print("Mean Absolute Error (MAE):", MAE_ridge)
print("Mean Percentage Error (MPE):", MPE_ridge)
print("Mean Absolute Percentage Error (MAPE):", MAPE_ridge)
print("\nModel Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

ridge_ME = ME_ridge
ridge_RMSE = RMSE_ridge
ridge_MAE = MAE_ridge
ridge_MPE = MPE_ridge
ridge_MAPE = MAPE_ridge
ridge_r_squared = r_squared
ridge_adjusted_r_squared = adj_r_squared
ridge_f_statistic = f_statistic
ridge_aic = aic
ridge_bic = bic

# Residual Analysis
residuals = y_test - y_pred_ridge

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_ridge, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Regularization Path
alphas = np.logspace(-6, 6, 200)
coefs = []
for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Ridge Regression Regularization Path')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_ridge, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Ridge Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()

Lasso regression (Least Absolute Shrinkage and Selection Operator) is a type of linear regression that adds an L1 regularization term to the cost function. This regularization penalizes the absolute values of the regression coefficients, forcing some of them to become exactly zero. As a result, lasso performs both feature selection and regularization, effectively simplifying the model by reducing the number of predictors. It is particularly useful when you have a large number of features, as it helps identify the most important ones while preventing overfitting.

Cross-validation is a statistical technique used to evaluate and improve the performance of a model by assessing how well it generalizes to unseen data. In the context of regression analysis, cross-validation is used to estimate the model’s performance on data that was not used to fit the model, helping to avoid overfitting.

Key Concepts:

	1.	Overfitting:
	•	Overfitting occurs when a model learns not only the underlying patterns in the training data but also the noise or random fluctuations. This leads to poor performance on unseen data.
	•	Cross-validation helps to avoid overfitting by testing the model on different subsets of the data, ensuring that it generalizes well.
	2.	Train-Test Split:
	•	In a basic model evaluation, you typically split the dataset into two parts: training data and testing data. The model is trained on the training data and evaluated on the testing data.
	•	However, this simple split can lead to issues because the model performance can be highly dependent on how the data was split.
	3.	K-Fold Cross-Validation:
	•	K-fold cross-validation is the most common form of cross-validation.
	•	The data is randomly split into K equal-sized subsets (folds).
	•	The model is trained on K-1 folds and tested on the remaining fold. This process is repeated K times, with each fold being used once as the testing set.
	•	The average performance across all K folds is then used as the final performance estimate of the model.
Example of 5-Fold Cross-Validation:
	•	Split the data into 5 parts (folds).
	•	Train the model on 4 parts and test it on the 5th part. Repeat this process 5 times (each time, a different part is used for testing).
  	4.	Leave-One-Out Cross-Validation (LOOCV):
	•	A special case of cross-validation where the number of folds is equal to the number of data points. Each time, the model is trained on all but one data point and tested on the left-out data point. This process is repeated for all data points.
	5.	Stratified Cross-Validation:
	•	In cases where the data has certain characteristics, such as imbalanced classes, stratified cross-validation ensures that each fold has approximately the same distribution of target variable values as the full dataset.

Steps in Cross-Validation for Regression:

	1.	Divide the data into K folds.
	2.	For each fold:
	•	Train the model on K-1 folds.
	•	Test the model on the remaining fold.
	3.	Calculate the performance metrics (e.g., mean squared error or R-squared) for each fold.
	4.	Average the performance metrics across all folds to get a final estimate of the model’s performance.

Why Use Cross-Validation in Regression?

	1.	Avoid Overfitting:
	•	Cross-validation helps ensure that your regression model is not just memorizing the training data but also generalizing well to new, unseen data.
	2.	Better Performance Estimates:
	•	A single train-test split might give a misleading performance estimate. Cross-validation provides a more reliable and stable estimate of model performance by evaluating the model on multiple subsets of data.
	3.	Hyperparameter Tuning:
	•	Cross-validation is often used in combination with hyperparameter tuning (e.g., choosing the best regularization parameter in Ridge or Lasso regression) to select the optimal model configuration.

In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Perform Lasso regression with Standard Scaler and alpha=0.5
lasso_model = make_pipeline(StandardScaler(), Lasso(alpha=1))
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)

# Calculate cross-validation scores
cv_scores = cross_val_score(lasso_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()

# Calculate validation errors for Lasso regression model
ME_lasso = np.mean(y_pred_lasso - y_test)
RMSE_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
MAE_lasso = mean_absolute_error(y_test, y_pred_lasso)
MPE_lasso = np.mean((y_pred_lasso - y_test) / y_test) * 100
MAPE_lasso = np.mean(np.abs((y_pred_lasso - y_test) / y_test)) * 100

# Get coefficients and corresponding variable names
coefficients = lasso_model.named_steps['lasso'].coef_
initial_variables = X_train.columns.tolist()

# Retained and dropped variables
retained_variables = [var for var, coef in zip(initial_variables, coefficients) if coef != 0]
dropped_variables = [var for var, coef in zip(initial_variables, coefficients) if coef == 0]

# Calculate VIF for each feature in the retained variables
X_train_retained = X_train[retained_variables]
X_train_const = sm.add_constant(X_train_retained)
vif_data = pd.DataFrame()
vif_data["Variable"] = X_train_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_const.values, i) for i in range(X_train_const.shape[1])]

# Fit a linear model using statsmodels to get p-values, R-squared, adjusted R-squared, AIC, BIC, and F-statistic
linear_model_sm = sm.OLS(y_train, X_train_const).fit()
p_values = linear_model_sm.pvalues
r_squared = linear_model_sm.rsquared
adj_r_squared = linear_model_sm.rsquared_adj
f_statistic = linear_model_sm.fvalue
aic = linear_model_sm.aic
bic = linear_model_sm.bic

# Create a DataFrame to display variable names, coefficients, p-values, and VIF
coefficients_full = [lasso_model.named_steps['lasso'].intercept_] + coefficients.tolist()
p_values_full = [None] + p_values.tolist()  # Add None for the intercept
coefficients_df = pd.DataFrame({
    'Variable': ['Intercept'] + retained_variables,
    'Coefficient': coefficients_full[:len(retained_variables) + 1],
    'P-Value': p_values_full[:len(retained_variables) + 1],
    'VIF': vif_data["VIF"]
})

# Print results
print("Dropped Variables:")
for variable in dropped_variables:
    print(variable)
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print("\nCross-Validation MSE:", cv_mse)
print("\nValidation Errors:")
print("Mean Error (ME):", ME_lasso)
print("Root Mean Square Error (RMSE):", RMSE_lasso)
print("Mean Absolute Error (MAE):", MAE_lasso)
print("Mean Percentage Error (MPE):", MPE_lasso)
print("Mean Absolute Percentage Error (MAPE):", MAPE_lasso)
print("\nModel Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

lasso_ME = ME_lasso
lasso_RMSE = RMSE_lasso
lasso_MAE = MAE_lasso
lasso_MPE = MPE_lasso
lasso_MAPE = MAPE_lasso
lasso_r_squared = r_squared
lasso_adjusted_r_squared = adj_r_squared
lasso_f_statistic = f_statistic
lasso_aic = aic
lasso_bic = bic


# Residual Analysis
residuals = y_test - y_pred_lasso

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_lasso, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Regularization Path
alphas = np.logspace(-6, 6, 200)
coefs = []
for a in alphas:
    lasso = Lasso(alpha=a)
    lasso.fit(X_train, y_train)
    coefs.append(lasso.coef_)

plt.figure(figsize=(10, 6))
plt.plot(alphas, coefs)
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Lasso Regression Regularization Path')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_lasso, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Lasso Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()

**Create and Elastic Net Model fitted with alpha=0.5 and l1_ratio=0.5**

lastic Net is a type of linear regression that combines both L1 (Lasso) and L2 (Ridge) regularization. It adds a penalty to the model's coefficients, balancing between feature selection (Lasso's strength) and coefficient shrinkage (Ridge's strength). This makes Elastic Net particularly useful when dealing with datasets that have many correlated predictors or when you want to benefit from both feature selection and regularization.

Key Parameters:
Alpha (α): Controls the overall strength of the regularization. A higher α increases the regularization effect, making the coefficients smaller and reducing overfitting.
L1 Ratio (ratio): Determines the balance between Lasso and Ridge regularization:
ratio = 1: Pure Lasso regularization (only L1 penalty).
ratio = 0: Pure Ridge regularization (only L2 penalty).
0 < ratio < 1: A combination of both Lasso and Ridge penalties.
Elastic Net is a flexible method that helps in scenarios where neither Lasso nor Ridge alone works perfectly, such as when you have multicollinearity or sparse data.








In [ ]:
# Define features and target
X = df_cleaned_columns.drop(columns=['Ug_enter'])
y = df_cleaned_columns['Ug_enter']

# Split the dataset into training (60%) and test sets (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=101)

# Perform Elastic Net regression with Standard Scaler, alpha=0.5, and l1_ratio=0.5
elastic_net_model = make_pipeline(StandardScaler(), ElasticNet(alpha=0.5, l1_ratio=0.5))
elastic_net_model.fit(X_train, y_train)
y_pred_en = elastic_net_model.predict(X_test)

# Calculate cross-validation scores
cv_scores = cross_val_score(elastic_net_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()

# Calculate validation errors for Elastic Net regression model
ME_en = np.mean(y_pred_en - y_test)
RMSE_en = np.sqrt(mean_squared_error(y_test, y_pred_en))
MAE_en = mean_absolute_error(y_test, y_pred_en)
MPE_en = np.mean((y_pred_en - y_test) / y_test) * 100
MAPE_en = np.mean(np.abs((y_pred_en - y_test) / y_test)) * 100

# Get coefficients and corresponding variable names
coefficients = elastic_net_model.named_steps['elasticnet'].coef_
initial_variables = X_train.columns.tolist()

# Retained and dropped variables based on Elastic Net coefficients
retained_variables = [var for var, coef in zip(initial_variables, coefficients) if abs(coef) > 1e-5]
dropped_variables = [var for var, coef in zip(initial_variables, coefficients) if abs(coef) <= 1e-5]

# Calculate VIF for each feature in the retained variables
def calculate_vif(X):
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

X_train_retained = X_train[retained_variables]
vif_data = calculate_vif(X_train_retained)

while vif_data['VIF'].max() > 10:
    var_to_drop = vif_data.loc[vif_data['VIF'].idxmax(), 'Variable']
    retained_variables.remove(var_to_drop)
    dropped_variables.append(var_to_drop)
    X_train_retained = X_train[retained_variables]
    vif_data = calculate_vif(X_train_retained)

# Fit a linear model using statsmodels to get p-values, R-squared, adjusted R-squared, AIC, BIC, and F-statistic
X_train_const = sm.add_constant(X_train_retained)
linear_model_sm = sm.OLS(y_train, X_train_const).fit()
p_values = linear_model_sm.pvalues
r_squared = linear_model_sm.rsquared
adj_r_squared = linear_model_sm.rsquared_adj
f_statistic = linear_model_sm.fvalue
aic = linear_model_sm.aic
bic = linear_model_sm.bic

# Ensure that lengths are aligned before creating the DataFrame
coefficients_full = [linear_model_sm.params[0]] + linear_model_sm.params[1:].tolist()
p_values_full = [None] + p_values[1:].tolist()  # Add None for the intercept
vif_full = [None] + vif_data["VIF"].tolist()

assert len(coefficients_full) == len(p_values_full) == len(vif_full) == len(['Intercept'] + retained_variables), "Array lengths do not match."

coefficients_df = pd.DataFrame({
    'Variable': ['Intercept'] + retained_variables,
    'Coefficient': coefficients_full,
    'P-Value': p_values_full,
    'VIF': vif_full
})

# Print results
print("Dropped Variables:")
for variable in dropped_variables:
    print(variable)
print("Coefficients, P-Values, and VIF:")
print(coefficients_df)
print()
print("Cross-Validation MSE:", cv_mse)
print()
print("Validation Errors:")
print("Mean Error (ME):", ME_en)
print("Root Mean Square Error (RMSE):", RMSE_en)
print("Mean Absolute Error (MAE):", MAE_en)
print("Mean Percentage Error (MPE):", MPE_en)
print("Mean Absolute Percentage Error (MAPE):", MAPE_en)
print()
print("Model Statistics:")
print("R-squared:", r_squared)
print("Adjusted R-squared:", adj_r_squared)
print("F-statistic:", f_statistic)
print("AIC:", aic)
print("BIC:", bic)

elastic_net_ME = ME_en
elastic_net_RMSE = RMSE_en
elastic_net_MAE = MAE_en
elastic_net_MPE = MPE_en
elastic_net_MAPE = MAPE_en
elastic_net_r_squared = r_squared
elastic_net_adjusted_r_squared = adj_r_squared
elastic_net_f_statistic = f_statistic
elastic_net_aic = aic
elastic_net_bic = bic


# Residual Analysis
residuals = y_test - y_pred_en

# Plot the residuals
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_en, residuals, color='blue', edgecolor='k', alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True)
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot')
plt.grid(True)
plt.show()

# Histogram of residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

# Regularization Path
alphas = np.logspace(-6, 6, 200)
coefs = []
for a in alphas:
    elastic_net = ElasticNet(alpha=a, l1_ratio=0.5)
    elastic_net.fit(X_train, y_train)
    coefs.append(elastic_net.coef_)

plt.figure(figsize=(10, 6))
for coef in np.array(coefs).T:
    plt.plot(alphas, coef)
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Elastic Net Regression Regularization Path')
plt.grid(True)
plt.show()

# Plot the regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_en, color='blue', edgecolor='k', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Elastic Net Regression: Actual vs Predicted Values')
plt.grid(True)
plt.show()

**Comparison of the Models**

In [ ]:
results = {
    'Model': ['Linear', 'Forward Selection', 'Backward Elimination', 'Ridge', 'Lasso', 'Elastic Net'],
    'RMSE': [liner_RMSE, forward_RMSE, backward_RMSE, RMSE_ridge, RMSE_lasso, RMSE_en],
    'MAE': [linear_MAE, forward_MAE, backward_MAE, MAE_ridge, MAE_lasso, MAE_en],
    'R-squared': [linear_r_squared, forward_r_squared, backward_r_squared, ridge_r_squared, lasso_r_squared, elastic_net_r_squared],
    'Adjusted R-squared': [linear_adjusted_r_squared, forward_adjusted_r_squared, backward_adjusted_r_squared, ridge_adjusted_r_squared, lasso_adjusted_r_squared, elastic_net_adjusted_r_squared],
    'AIC': [linear_aic, forward_aic, backward_aic, ridge_aic, lasso_aic, elastic_net_aic],
    'BIC': [linear_bic, forward_bic, backward_bic, ridge_bic, lasso_bic, elastic_net_bic]
}

# Create a DataFrame to store the results
results_df = pd.DataFrame(results)

# Adjust display settings to avoid wrapping columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Print the DataFrame
print("Comparison of Regression Models:")
print(results_df)

**Best Fit Model**

In [ ]:
# Function to determine the best model based on given criteria
def find_best_model(results_df):
    # Rank models based on each metric
    results_df['RMSE_rank'] = results_df['RMSE'].rank(ascending=True)
    results_df['MAE_rank'] = results_df['MAE'].rank(ascending=True)
    results_df['R-squared_rank'] = results_df['R-squared'].rank(ascending=False)
    results_df['Adjusted_R-squared_rank'] = results_df['Adjusted R-squared'].rank(ascending=False)
    results_df['AIC_rank'] = results_df['AIC'].rank(ascending=True)
    results_df['BIC_rank'] = results_df['BIC'].rank(ascending=True)

    # Calculate total rank (lower is better)
    results_df['Total_rank'] = results_df[['RMSE_rank', 'MAE_rank', 'R-squared_rank', 'Adjusted_R-squared_rank', 'AIC_rank', 'BIC_rank']].sum(axis=1)

    # Find the best model
    best_model = results_df.loc[results_df['Total_rank'].idxmin()]

    return best_model

# Find the best model
best_model = find_best_model(results_df)

print("Best Model:")
print(best_model)